# Lab 7: P-Tuning - Fine-Tuning a BERT Model for NLU Tasks
---
## Notebook 3: Inference

**Goal:** In this notebook, you will load the trained P-Tuning adapter and use the fine-tuned BERT model to perform sentiment classification.

**You will learn to:**
-   Reload the base BERT model and tokenizer.
-   Load the trained P-Tuning adapter from a checkpoint.
-   Perform sentiment classification on new sentences.


### Step 1: Reload Model and Adapter

We will load the base BERT model and then apply our trained P-Tuning weights on top of it.


In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from peft import PeftModel
import torch
import os

# --- Load Base Model and Tokenizer ---
model_checkpoint = "bert-base-uncased"
base_model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint, 
    num_labels=2
)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

# --- Load PEFT Adapter ---
output_dir = "./bert-ptuning-sst2"
latest_checkpoint = max(
    [os.path.join(output_dir, d) for d in os.listdir(output_dir) if d.startswith("checkpoint-")],
    key=os.path.getmtime
)
print(f"Loading adapter from: {latest_checkpoint}")

inference_model = PeftModel.from_pretrained(base_model, latest_checkpoint)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
inference_model.to(device)
inference_model.eval()

print("✅ P-Tuning fine-tuned model loaded successfully!")


### Step 2: Perform Sentiment Classification

Now, let's test the model by performing sentiment classification on new sentences. We'll use the same prediction function approach as in other labs.


In [ ]:
import torch.nn.functional as F

# Define the labels
id2label = {0: "Negative", 1: "Positive"}

def predict_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    with torch.no_grad():
        outputs = inference_model(**inputs)
    
    logits = outputs.logits
    probabilities = F.softmax(logits, dim=1).cpu().numpy()[0]
    prediction = torch.argmax(logits, dim=-1).cpu().item()
    
    print(f"Text: '{text}'")
    print(f"Prediction: {id2label[prediction]}")
    print(f"Probabilities:")
    print(f"  - {id2label[0]}: {probabilities[0]:.4f}")
    print(f"  - {id2label[1]}: {probabilities[1]:.4f}")
    print()

# --- Test Cases ---
test_sentences = [
    "This movie was absolutely fantastic! I loved every minute of it.",
    "The film was boring and poorly made. I didn't enjoy it at all.",
    "An average movie with some good moments.",
    "Outstanding performance by the lead actor. Highly recommended!",
    "Terrible plot and bad acting. Complete waste of time."
]

print("--- P-Tuning Sentiment Classification Results ---")
for sentence in test_sentences:
    predict_sentiment(sentence)
